Programming Assignment: Анализ текстов

Вы научитесь:

находить оптимальные параметры для метода опорных векторов
работать с текстовыми данными

In [88]:
import numpy as np
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
#from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC

newsgroups = datasets.fetch_20newsgroups(
    subset='all', 
    categories=['alt.atheism', 'sci.space']
)

# Объявляется векторизатор и данные трансформируются в набор векторов
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(newsgroups.data)
y = newsgroups.target

# Выделяется набор слов соответсвующих признакам по идентификатору
feature_mapping = vectorizer.get_feature_names()

# Создается набор вариантов для параметра алгоритма C
grid = {'C': np.power(10.0, np.arange(-5, 6))}   # в виде словаря параметров для использования в GridSearchCV
# C_svm = map(lambda x: 10**x, range(-5,6))   # в виде списка

cv = KFold(n_splits=5, shuffle=True, random_state=241)    # Создается рабивка данных для кросс-валидации
clf = SVC(kernel='linear', random_state=241)    # Объявление классификатора SVM, параметр С будет автоматически подбираться
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv) 
gs.fit(vectors, y)    # Подбираем параметры

print("Best parameters set found on all set")
print()
print(gs.best_params_)
print()
print("Grid scores on set:")
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
parametrs = gs.cv_results_['params']
for mean, std, params in zip(means, stds, parametrs):
    print("%f %f for %r" % (mean, std, params))

Best parameters set found on all set
()
{'C': 1.0}
()
Grid scores on set:
0.552632 0.028117 for {'C': 1.0000000000000001e-05}
0.552632 0.028117 for {'C': 0.0001}
0.552632 0.028117 for {'C': 0.001}
0.552632 0.028117 for {'C': 0.01}
0.950168 0.008218 for {'C': 0.10000000000000001}
0.993281 0.004551 for {'C': 1.0}
0.993281 0.004551 for {'C': 10.0}
0.993281 0.004551 for {'C': 100.0}
0.993281 0.004551 for {'C': 1000.0}
0.993281 0.004551 for {'C': 10000.0}
0.993281 0.004551 for {'C': 100000.0}


In [90]:
clf = SVC(C=1, kernel='linear', random_state=241)    # Обучаем на первом лучшем значении параметра С
clf.fit(vectors, y)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [92]:
clf.coef_.shape

(1, 28382)

In [169]:
len(feature_mapping)

28382

In [225]:
# Вычисляем 10 слов с наибольшим абсолютным значением веса
# По сути данным образом мы смотрим 10 признаков имеющих наибольший вес в классификации
# В данном случае слова у нас выступают в качестве признаков

d = {}
for coef, feat in zip(w, feature_mapping):
    d[feat] = coef
    
print clf.coef_[0, 28381]
#print clf.coef_[0]

for index in range(len(feature_mapping)):
    d[feature_mapping[index]] = abs(clf.coef_[0, index])
    
print d

-0.00297347158513
{u'schlegel': 0.0, u'luanch': 0.0, u'tilton': 0.0088589887233260656, u'woods': 0.09841374733157035, u'hanging': 0.035118376826684755, u'localized': 0.020394691672327832, u'disobeying': 0.038234550782557226, u'734841689': 0.0, u'fractioning': 0.019441004969437067, u'5980': 0.025897404061804765, u'rickman': 0.016499717271406082, u'pigment': 0.0, u'bringing': 0.090581342520733327, u'vibrational': 0.062457869866070284, u'wednesday': 0.087249682576154328, u'viable': 0.0075191572144384326, u'matthean': 0.0, u'270': 0.0047118141276138188, u'271': 0.0, u'272': 0.0, u'273': 0.0, u'274': 0.020231056716022426, u'275': 0.015894126122225748, u'276': 0.031894365197850716, u'277': 0.0, u'scraped': 0.055437841844785653, u'inanimate': 0.019946569613260762, u'errors': 0.17740710856103978, u'cooking': 0.053960356817295621, u'usenet': 0.060324755932906649, u'100010': 0.02655330936647534, u'designing': 0.023985240260349136, u'numeram': 0.0, u'replaced': 0.016937363761196811, u'hallucinati

In [226]:
# Сортируем словарь "слово: значение веса" по весу по убыванию
import operator

sorted_dict = sorted(d.items(), key=operator.itemgetter(1), reverse=True)

In [241]:
result = []
for i in range(10):
    print sorted_dict[i]
    result.append(sorted_dict[i][0])

(u'space', 2.6631647884797105)
(u'god', 1.9203794002294938)
(u'atheism', 1.2546899512384038)
(u'atheists', 1.2491800073760075)
(u'moon', 1.2016111817520696)
(u'sky', 1.1801315951388633)
(u'religion', 1.13908083789883)
(u'bible', 1.130612344664901)
(u'keith', 1.0970936466401482)
(u'sci', 1.0293069271856938)


In [245]:
result.sort()
result

[u'atheism',
 u'atheists',
 u'bible',
 u'god',
 u'keith',
 u'moon',
 u'religion',
 u'sci',
 u'sky',
 u'space']